In [1]:
import pandas

### Using magic commands to get the dir the file is in

In [2]:
basepath = %pwd

### import dataset


In [3]:
baseCsv = pandas.read_csv(basepath + '/raw/crash_data.csv', sep=',', index_col=False, low_memory=False)

### Make Report Number Unique

In [16]:
baseCsv['Report Number'] = baseCsv['Report Number'].astype(str).str.strip()
baseCsv = baseCsv.drop_duplicates(subset=['Report Number'], keep='first').reset_index(drop=True)


Get Null precentage for each colum in the dataset

In [17]:
NullPrecentage = baseCsv.isnull().sum() * 100 / len(baseCsv)
NullPrecentage

Report Number                      0.000000
Local Case Number                  0.000000
Agency Name                        0.000000
ACRS Report Type                   0.000000
Crash Date/Time                    0.000000
Route Type                        13.114585
Road Name                         12.429017
Cross-Street Type                 13.134202
Cross-Street Name                 12.439342
Off-Road Description              87.573048
Municipality                      89.217792
Related Non-Motorist              94.540236
Collision Type                     0.504884
Weather                            8.102918
Surface Condition                 14.803725
Light                              0.825986
Traffic Control                   16.703492
Driver Substance Abuse            18.020939
Non-Motorist Substance Abuse      95.674933
Person ID                          0.000000
Driver At Fault                    0.000000
Injury Severity                    0.000000
Circumstance                    

# Removing Colums with more than 60% Null Values
## Following the Business Model

In [18]:
ColumnsToDrop = NullPrecentage[NullPrecentage > 60].index
if not ColumnsToDrop.empty:
 baseCsv = baseCsv.drop(ColumnsToDrop, axis=1)
baseCsv.dtypes

Report Number                      object
Local Case Number                  object
Agency Name                        object
ACRS Report Type                   object
Crash Date/Time                    object
Route Type                         object
Road Name                          object
Cross-Street Type                  object
Cross-Street Name                  object
Collision Type                     object
Weather                            object
Surface Condition                  object
Light                              object
Traffic Control                    object
Driver Substance Abuse             object
Person ID                          object
Driver At Fault                    object
Injury Severity                    object
Driver Distracted By               object
Drivers License State              object
Vehicle ID                         object
Vehicle Damage Extent              object
Vehicle First Impact Location      object
Vehicle Second Impact Location    

In [19]:
baseCsv.isnull().sum()

Report Number                         0
Local Case Number                     0
Agency Name                           0
ACRS Report Type                      0
Crash Date/Time                       0
Route Type                        12702
Road Name                         12038
Cross-Street Type                 12721
Cross-Street Name                 12048
Collision Type                      489
Weather                            7848
Surface Condition                 14338
Light                               800
Traffic Control                   16178
Driver Substance Abuse            17454
Person ID                             0
Driver At Fault                       0
Injury Severity                       0
Driver Distracted By                  0
Drivers License State              6891
Vehicle ID                            0
Vehicle Damage Extent               199
Vehicle First Impact Location        90
Vehicle Second Impact Location      140
Vehicle Body Type                  1847


# Removing `Yes,No,Nulls` into valid Boolean for use
## Replacing `Nulls` in the rest with the most frequent value found in that column 

In [20]:
baseCsv['Driverless Vehicle'] = baseCsv['Driverless Vehicle'].str.lower()
baseCsv['Driverless Vehicle'] = baseCsv['Driverless Vehicle'].replace({
    'yes': True,
    'no': False,
    'unknown': False
})

baseCsv['Parked Vehicle'] = baseCsv['Parked Vehicle'].str.lower()
baseCsv['Parked Vehicle'] = baseCsv['Parked Vehicle'].replace({
    'yes': True,
    'no': False,
    'unknown': False
})

baseCsv['Driver Substance Abuse'] = baseCsv['Driver Substance Abuse'].str.lower()
baseCsv['Driver Substance Abuse'] = baseCsv['Driver Substance Abuse'].replace({
    'yes': True,
    'no': False,
    'unknown': False
})

baseCsv['Driver At Fault'] = baseCsv['Driver At Fault'].str.lower()
baseCsv['Driver At Fault'] = baseCsv['Driver At Fault'].replace({
    'yes': True,
    'no': False,
    'unknown': False
}).astype('boolean')

baseCsv = baseCsv.apply(
    lambda col: col.fillna(
        col.loc[
            ~col.str.lower().isin(['unknown', 'n/a'])
        ].mode()[0] if col.dropna().size > 0 and not col.loc[
            ~col.str.lower().isin(['unknown', 'n/a'])
        ].mode().empty else col.fillna('N/A')
    ) 
    if col.dtype == 'object' else col.fillna(0), 
    axis=0
)

/tmp/ipykernel_43078/809176932.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  baseCsv['Driverless Vehicle'] = baseCsv['Driverless Vehicle'].replace({
/tmp/ipykernel_43078/809176932.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  baseCsv['Parked Vehicle'] = baseCsv['Parked Vehicle'].replace({
/tmp/ipykernel_43078/809176932.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to

In [21]:
baseCsv.isnull().sum()

Report Number                     0
Local Case Number                 0
Agency Name                       0
ACRS Report Type                  0
Crash Date/Time                   0
Route Type                        0
Road Name                         0
Cross-Street Type                 0
Cross-Street Name                 0
Collision Type                    0
Weather                           0
Surface Condition                 0
Light                             0
Traffic Control                   0
Driver Substance Abuse            0
Person ID                         0
Driver At Fault                   0
Injury Severity                   0
Driver Distracted By              0
Drivers License State             0
Vehicle ID                        0
Vehicle Damage Extent             0
Vehicle First Impact Location     0
Vehicle Second Impact Location    0
Vehicle Body Type                 0
Vehicle Movement                  0
Vehicle Continuing Dir            0
Vehicle Going Dir           

### Split the dataset into 5 small ones

In [22]:
# Group 1: Report and Case Information
df_report_case = baseCsv[['Report Number', 'Local Case Number', 'Agency Name', "ACRS Report Type", "Crash Date/Time", 'Location']]
df_report_case.insert(0, 'id', range(1, 1 + len(df_report_case)))

# Group 2: Road and Location Information 
df_road_location = baseCsv[['Route Type', 'Road Name', 
                       'Cross-Street Name', 'Latitude', 'Longitude']]
df_road_location.insert(0, 'id', range(1, 1 + len(df_road_location)))


# Group 3: Vehicle Information 
df_vehicle = baseCsv[['Vehicle ID', 'Vehicle Damage Extent', 
                 'Vehicle First Impact Location', 'Vehicle Second Impact Location', 
                 'Vehicle Body Type', 'Vehicle Movement', 'Vehicle Continuing Dir', 
                 'Vehicle Going Dir', 'Speed Limit', 'Driverless Vehicle', 
                 'Parked Vehicle', 'Vehicle Year', 'Vehicle Make', 'Vehicle Model', 
                 'Equipment Problems']]
df_vehicle.insert(0, 'id', range(1, 1 + len(df_vehicle)))

# Group 4: Driver and Person Information
df_driver_person = baseCsv[['Person ID', 'Driver At Fault', 
                       'Driver Substance Abuse', 'Driver Distracted By', 
                       'Drivers License State']]
df_driver_person.insert(0, 'id', range(1, 1 + len(df_driver_person)))


# Group 5: Incident Details 
df_incident_details = baseCsv[['Collision Type', 'Weather', 'Surface Condition', 
                          'Light', 'Traffic Control', 'Injury Severity']]
df_incident_details.insert(0, 'id', range(1, 1 + len(df_incident_details)))


### Save the split datasets 

In [23]:
df_report_case.to_csv(basepath + '/loaded/report_case.csv', index=False)
df_road_location.to_csv(basepath + '/loaded/road_location.csv', index=False)
df_vehicle.to_csv(basepath + '/loaded/vehicle.csv', index=False)
df_driver_person.to_csv(basepath + '/loaded/driver_person.txt', sep=',', index=False, header=True)
df_incident_details.to_csv(basepath + '/loaded/incident_details.txt', sep=',', index=False, header=True)